# Decoding translation in the cloud and at NCBI

** Exercise for Upstate Summer Career Awareness Day **

---

### Run some Python code in a Jupyter notebook!

To run the code below:

1. Click on each cell to select it.
2. Press `SHIFT+ENTER` on your keyboard or press the play button (<button class='fa fa-play icon-play btn btn-xs btn-default'></button>) in the toolbar above.

Attributions: Below borrows in part from [Illustrating Python via Bioinformatics Examples (Bioinf-py)](http://hplgit.github.io/bioinf-py/doc/web/index.html).

Before beginning the exercise proper, I'd like to point out this notebook itself is hosted on a remote server and powered by cloud computing. You can see this by entering a unix shell command below to see the directory of that machine. Jupyter notebooks allow accessing the host computer's shell via magic commands proceeded by '`!`'.  Click on the next cell and press `SHIFT+ENTER` on your keyboard to reveal this.


In [ ]:
!ls -lah

Now step through the cells below to run the code for the exercise.

---

### Define a function to generate sequences

Understanding how often to expect things, such as restriction endonuclease recognition sites or transcription factor binidng sites, in a sequence of nucleic acid is useful for planning and interpreting experiments. Because we still don't have everything sequenced,  we often will rely on what are called random sequences to understand probability of an occurence as it gives us at least a baseline impression of the frequency, high or low, of an event. In one of the excericises today we found a DNA sequence that translated to some meaningful text. That situation was artificially contrived as an exercise, but we might ask how frequent certain strings of characters occur among the protein sequences. For the sake of the exercise, we'll stay with string of characters that are meaningful as common text. Is it rare or common for "PIDGEY" to be represented among protein sequences? How about your first name? How about your first and last name?

We'll build to be able to perform this endeavor in steps. The first step is to define a function that will generate a random DNA sequence of the length we define.  Click on the next cell and type `SHIFT+ENTER` on your keyboard to run the code to define that function.

In [ ]:
import random # We need an additional package that isn't part of base Python.
# The next line is where the function gets define. This is a comment line.

def generate_DNA(N):
    possible_bases ='ACGT'
    return ''.join([random.choice(possible_bases) for i in range(N)])

Okay, you pressed `SHIFT+ENTER` on your keyboard while selecting the above cell but nothing impressive happened, right?  
It only seemed nothing happened. Behind the scenes the code ran properly and a function has been defined. Next we need to invoke that function by calling it with the length we want. Let's call the function and assign it to a variable. Then we can print the result by calling a `print` function with that variable as an argument. The `print` function is built in to base Python.

In [ ]:
my_first_dna_sequence = generate_DNA(100)
print (my_first_dna_sequence)

You should see a sequence of 100 bps in length. (Only one strand is shown and the complementary pair is implied.)  
If you pressed `SHIFT+ENTER` on your keyboard while selecting the above cell and got an error, most likely you had not defined the `generate_DNA` function. Go do that and then try again.

We can easily adjust the length by changing the argument we call the function with. Let's try `2000`.

In [ ]:
two_kb_sequence = generate_DNA(2000) #kb stands for kilobase pairs.
# One quirk of python is that variable names cannot begin with numbers. 
# Hence the `two` and not `2`.

print (two_kb_sequence)

### Generate a lot of sequences

Now we have a function to generate a simulated random DNA sequence. And because programing allow you to easily do one thing over and over, we can get a large collection now easily.

Let's generate a 100 sequences all of 1000 basepair length. We will make a `for` loop to iterate over our function for 100 times. We will make a list of our sequences as we do this.

In [ ]:
sequence_list = [] # We initialize our list of sequences.

for i in range(100):
    one_kb_sequence = generate_DNA(1000)
    sequence_list.append(one_kb_sequence)
    

Now you could use the `print` function to see the list. Feel free to insert a cell and write the code to do that. However, since printing the list will generate long hard to view list, let's assess the activity of our code another way.

We'd expect the sequence of each base to be represented about a quarter of the time. So let's count the occurence of `G` in each sequence and plot it.

First we have to set up the notebook to do that by issuing some commands and importing more packages that are not part of the base Python language. Just `shift + enter` on the next cell, pay no heed to any pink text generated, and skip onto the more interesting stuff after.

In [ ]:
%matplotlib inline
# technical trick to get plots below to be displayed in this notebook,
# see http://stackoverflow.com/questions/26597116/seaborn-plots-not-showing-up

import seaborn as sns # seaborn is a package that makes it easy to draw 
# attractive statistical graphics.

With the technical step done, let's count the occurences of G's and plot.

In [ ]:
g_counts = [sequence.count('G') for sequence in sequence_list] 
sns.distplot(g_counts)

You may see a bell-shaped curve centered on `250`. More than likely it doesn't look perfect. One hundred sequences is still a small sample.  
Let's  jump up two orders of magnitude to ten thousand sequences, and then see if the curve looks smoother and more like the normal distribution it should represent.

In [ ]:
sequence_list = [] # We reinitialize our list of sequences to clear old.

for i in range(10000):
    one_kb_sequence = generate_DNA(1000)
    sequence_list.append(one_kb_sequence)

Watch for how long it takes the asterisk `*` to go away in between the brackets next to the `In` after pressing `Shift+ Enter` to run the cell above. (Feel free to do it again, if you missed it.) This delay is because it takes some time to make and store ten thousand sequences. Python is not the fastest programming language out there, but being higher level it is one of those that is appears most like normal language, and thus tends to be easier to learn.  
Generating one hundred thousand would be longer still, and one million would take much longer still. (You can try those later and see for yourself, if you'd like.)

Let's plot the new results.

In [ ]:
g_counts = [sequence.count('G') for sequence in sequence_list] 
sns.distplot(g_counts)

Yes, we indeed smoothed our distribution by sampling more.

Better yet, our function indeed seems to generate random sequences, and we can now easily make a list of them to use for the next part of the exercise.

### Make messenger RNA from the DNA

We need to convert the DNA sequences we have now represented as strings of bases together to RNA. Concerning the bases, the difference between DNA and RNA is that '`T`' is replaced by '`U`'.  
Let's make the replacement in our sequences to reflect that. We'll use what is called a `list comprehension` in Python to perform an action to every member of a list. (We actually used a list comprehension for making the counts above too.) The `for` loop we've seen before is now concisely on one line with the action we want to take place at the start of the command, and since it results in a list again it is in brackets.

In [ ]:
sequence_list = [sequence.replace('T', 'U') for sequence in sequence_list] 

Now we have sequences representing random messenger RNA sequences that we can decode and examine for patterns of character strings.

### Translating "gene" sequences to protein sequences

The genetic code is what dictates how gene sequences are read and utilized to make protein sequences in the process of transcription and translation. We are now at the point we want to simulate translation and so we need to incorporate a representation of translation. We will do this with a Python dictionary. Each codon will be a key and assigned a corresponding value. In this case the value for each an amino acid that the particular codon gets translated into. Each amino acid in a protein sequence has a full name, such as Methionine, Tryptophan, and Alanine. Because the codon sequence is based on three bases, each amino acid has a three-letter code that represents it so it makes it easy to match each codon to the three-letter amino acid code as you are translating by hand or representing the translated results below a gene sequence in word processing document or figure. In order to condense the text needed to represent the protein sequence further, there is a single-letter abbreviation for amino acid. We'll construct our dictionary using the one-letter code for each amino acid since it is what we ultimately want to use to look for patterns of letters in anyway. Let's set that up. 

In [ ]:
genetic_code_dict = {
     'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L', 'UCU': 'S',
     'UCC': 'S', 'UCA': 'S', 'UCG': 'S', 'UAU': 'Y', 'UAC': 'Y',
     'UGU': 'C', 'UGC': 'C', 'UGG': 'W', 'CUU': 'L', 'CUC': 'L',
     'CUA': 'L', 'CUG': 'L', 'CCU': 'P', 'CCC': 'P', 'CCA': 'P',
     'CCG': 'P', 'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
     'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AUU': 'I',
     'AUC': 'I', 'AUA': 'I', 'AUG': 'M', 'ACU': 'T', 'ACC': 'T',
     'ACA': 'T', 'ACG': 'T', 'AAU': 'N', 'AAC': 'N', 'AAA': 'K',
     'AAG': 'K', 'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
     'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V', 'GCU': 'A',
     'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAU': 'D', 'GAC': 'D',
     'GAA': 'E', 'GAG': 'E', 'GGU': 'G', 'GGC': 'G', 'GGA': 'G',
     'GGG': 'G', 'UAA': '*', 'UAG': '*', 'UGA': '*'}
# We'll use asterisks to represent the stop codons that signal the 
# end of translation.


Now we are ready to write a function to simulate translation. The first draft is below. Run the code and then below we'll discuss it.

In [ ]:
def create_protein_first_attempt(mRNA):
    protein = ''
    for i in range(len(mRNA)/3):
        codon_start = i * 3
        end = start + 3
        protein += genetic_code_dict[mRNA[codon_start:end]]
    return protein

# We'd invoke it like on the next line but we aren't ready to do that yet,
# and so it is commented out.
# protein_sequence = create_protein(mRNA)

This function synthesizes a lot of what we already covered about Python programming and adds some more. The main thing that it is added is `slicing`. That is represented here on the 6th line where you see `mRNA[codon_start:codon_end]`. It means take a slice of the string starting at the position defined by start and going up to but not including three characters after the first, i.e., get three letters from the sequence, or a codon. Here it is used to slice a sequence of characters, a string, but you can slice other data forms, such as the lists we have seen before. Soon we will use that ability to pull out the first few sequences in our sequence list and try our function on it. Python will slice at the highest informational content unit it recognizes. In our `sequence_list`, that unit is each sequence of letters. In the function we just defined, it is slicing a sequence of letters up into subsets of letters because that is the highest unit that exists. This slicing ability makes Python great for handling text which is the major way to represent genetic sequence information.

**Unfortunately, this first try to simulate the translation process is incorrect.** The problem is that translation (almost) always begins with the amino acid Methionine, code '`AUG`', and ends when a stop codons is encounyered. We must thus check for the correct start and stop criteria. A fix is

In [ ]:
def create_protein(mRNA):
    protein = ''
    trans_start_pos = mRNA.find('AUG')
    for i in range(len(mRNA[trans_start_pos:])//3):
        codon_start = trans_start_pos + i*3
        end = codon_start + 3
        amino = genetic_code_dict[mRNA[codon_start:end]]
        if amino == '*':
            break
        protein += amino
    return protein

# We'd invoke it like on the next line but we aren't ready to do that yet,
# and so it is commented out.
# protein_sequence = create_protein(mRNA)

The above function has further built on what we know of Python. Now we added on the 8th line an '`if`' conditional test. We do something only if the condition is met. This is a way to start branching the action of your code dependent on conditions at the time.

Let's go ahead and try this function by slicing out the first few sequence from our sequence list and have them translated. Here, I will let you in on another quirk of Python. It is zero-indexed, meaning, the first item in any list is not at position `one` but is at the `zero` position. It is a difficult concept for everyone used to number one be synomous with the first position but there it is.   

In [ ]:
for sequence in sequence_list[0:10]:
    protein_sequence = create_protein(sequence)
    print (protein_sequence)

Hopefully you will at least see some protein sequences generated, and thus the simulation works in a rudimentary fashion. There is definitely room for improvement. One caveat is that as written it is only finding the first '`AUG`' in each sequence and working only from that. You might imagine it would be better to look at all the possible open reading frames in each sequence. Other caveats are mentioned below. 

If you get an error about the `genetic_code_dict`, make sure you go select the cell where you defined it and hit `Shift + Enter` to run that code so the dictionary is defined. Then try again.

If you get an `index out of range error`, it may just mean a sequence happened to lack an '`AUG`'. Since this is random sequences, it could happen. (In the course of developing the code, it seemed rare.) No big deal. Just change the '`0'` and '`10`' to be any other numbers greater than 0 and less than 10,000.

### Searching the protein sequences for matches

So far we just printed some sequences. We need to collect them all. And then look for our string of characters of interest. We'll loop through and collect them like we did for our '`sequence_list`'.

In [ ]:
protein_list = [] # initialize a list for collecting

for sequence in sequence_list:
    protein_sequence = create_protein(sequence)
    protein_list.append(protein_sequence)

After running the above cell, we'll have a list of protein sequences we can search and assign to a new list we'll call '`matching`'.

(I should point out that we just did what one of the exercises from the hands-on exercises involves, but we did it 10,000 times in an instant. Programming lets you do repetitive steps much more efficiently and it it is a good fit for research because you will need to redo things over and over. We often joke that is why it is called '`re`'-search.)

In [ ]:
string_to_find = "PIDGEY"

matching = [protein for protein in protein_list if string_to_find.upper() in protein]
# the `.upper()` function call insures the matching will be case-insensitive

In [ ]:
print (matching)

Be prepared to see no matches between the brackets.

If no matches, remove one of the letters and run the last two cells again. In my experience while developing this exercise, I was able to regularly get matches to "`IDGEY`". Once I did get a match to "PIDGEY" when testing after clearing the output and running all the cells again. You can try that a few times by going to the '`Kernel`' menu above and selecting '`Restart & Run All`'.  (Alternatively, under the'`Cell`' menu and you can select '`All Output`' > '`Clear`' and then also from the '`Cell`' menu, select '`Run All`'.) 

Try changing the string to something else.  
Why can't we search for "WALDO"?

Obviously, this code was optimized to run in a reasonable amount of time and not exceed the limits for the free service. Thus, concessions were made and it has it's limits. The point is that given this small sample, it is rare to find a string of any significant length among random sequences.  
Despite the concessions, the exercise provides a step-by-step coverage of the process and introduced Python.  Understanding the process was the goal here. We could always run the code elsewhere for a longer time, i.e., with more and longer "gene" sequences, to push the limits more. 

**Caveats:**  
- To make things easy, we were only going to concerned with decoding the strand we made directly. Potentially there are 3 possible reading frames on each strand of double-stranded DNA, for a total of 6 possible frames. Plus,  we didn't discuss antisense, splicing, the issue that not every '`AUG`' is a start codon, a sequence can harbor multiple open reading frames, alternative genetic codes, etc. 
- 1 kb is a small segment to represent as containing gene. Average gene size in humans is 10 - 15 kb. The average is 1.6 kb in yeast. One kb wouldn't contain those.
- Of course, nature isn't random. One way it isn't random is that the genomes of organisms have particular %GC content that are seldom 50% like we used in our simulation. 

### Things get real

Now that we know better what to expect for probabilities in the case of sequences of proteins generated from random nucleic acid sequences, what if we look for string patterns in real protein sequences. This also has the advantage that the sequences are already generated and greatly exceed what we could generate in a short amount of time (and within the computational limits of a free service) above.

Go to [here](http://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE=Proteins) and enter "PIDGEY". What do you see?
What about with "MEDICALRESEARCH"? 
Feel free to try other sequences of letters.

---

### Conclusion

Hopefully, you've seen some of how some basic computer skills can allow you to work efficiently and accomplish a lot. And maybe you've found how a high level programming language like Python might be a good choice to learn? You've gotten a little experience using the National Center for Biotechnology Information search tool. Plus maybe by using the Jupyter notebook you've glimpsed the direction of the open source and scientific computing fields in the near future.

You can always start this notebook again at a future date by going to [bit.ly/scadpython](http://bit.ly/scadpython) and pressing the '`Launch Binder`' button. Thanks.